In [ ]:
!git clone https://github.com/ks-1180/art-style-challenge.git
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

need this for styleGAN image generation for plugins: "bias_act_plugin" and "upfirdn2d_plugin"

In [ ]:
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force
!rm ninja-linux.zip

In [ ]:
!pip install gradio

In [5]:
import gradio as gr
import os
import sys
# TODO: change to repo after finishing
%cd /content/stylegan2-ada-pytorch

library_paths = ['.']

import dnnlib, legacy

for p in library_paths:
    if p not in sys.path:
        sys.path.append(p)

# pls update root accordingly
%cd /content/art-style-challenge/src

#from art-style-challenge import evaluator
import generator
import projector
from align import FaceDetector

import matplotlib.pyplot as plt
import cv2
import PIL.Image
import torch

/content/stylegan2-ada-pytorch
/content/art-style-challenge/src


# Setup

download all the models

In [6]:
resolution = 256
base_model = 'https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/transfer-learning-source-nets/ffhq-res256-mirror-paper256-noaug.pkl'

In [7]:
import os
import urllib

models = {
    'models_male': {
        'painting_male.pkl': 'https://owncloud.tuwien.ac.at/index.php/s/CEcsQYiXLrIpDxv/download',
        'digital_art_male.pkl': 'https://owncloud.tuwien.ac.at/index.php/s/U35bDm6bWGpDA0U/download',
        'fire_emblem_male.pkl': 'https://owncloud.tuwien.ac.at/index.php/s/RYx1SScuRgqYIVk/download',
        'ghibli_male.pkl': 'https://owncloud.tuwien.ac.at/index.php/s/WRlwHXCVqX2zxgA/download',
        'naruto_male.pkl': 'https://owncloud.tuwien.ac.at/index.php/s/Mx44GVUWCsZHea2/download',
        'one_piece_male.pkl': 'https://owncloud.tuwien.ac.at/index.php/s/GpPwqLrJYSpgIml/download',
    },
    'models_female': {
        'digital_art_female.pkl': 'https://owncloud.tuwien.ac.at/index.php/s/SXM0t7sfJGYfG3X/download',
        'painting_female.pkl': 'https://owncloud.tuwien.ac.at/index.php/s/ps9gV9oxOQuT4Ax/download',
        'fire_emblem_female.pkl': 'https://owncloud.tuwien.ac.at/index.php/s/59bviBNoxTUlEQL/download',
        'ghibli_female.pkl': 'https://owncloud.tuwien.ac.at/index.php/s/BSjoPwd23Q52DtM/download',
        'naruto_female.pkl': 'https://owncloud.tuwien.ac.at/index.php/s/ehHYVTnpqXg1g3V/download',
        'one_piece_female.pkl': 'https://owncloud.tuwien.ac.at/index.php/s/ASPcSSDipPgr4ye/download',
    }
}

for folder, content in models.items():
  os.makedirs(f'/content/{folder}', exist_ok=True)
  for name, url in content.items():
    if not os.path.exists(f'/content/{folder}/{name}'):
      print(f'Downloading {name}...')
      urllib.request.urlretrieve(url, f'/content/{folder}/{name}')
    else:
      print(f'/content/{folder}/{name} already exists, skipping...')

In [8]:
style_models_f = '/content/models_female'
style_models_m = '/content/models_male'

In [9]:
gen_female = generator.Generator(base_model, style_models_f, resolution)

In [10]:
gen_male = generator.Generator(base_model, style_models_m, resolution)

In [12]:
face_detector = FaceDetector()

RuntimeError: ignored

execute once to setup: "bias_act_plugin" and "upfirdn2d_plugin"

# Gradio App

In [11]:
#from IPython.lib.security import passwd
import numpy as np
from PIL import Image
import time

def gen_image(image, features, iterations, dropdown):

  #for _ in range(2):
  #    time.sleep(1)
  #    image = np.random.random((600, 600, 3))
  #    yield image

  #image = "https://i.picsum.photos/id/867/600/600.jpg?hmac=qE7QFJwLmlE_WKI7zMH6SgH5iY5fx8ec6ZJQBwKRT44"
  #yield image
  
  gen = gen_female if features == 'feminine' else gen_male
  append = '_female.pkl' if features == 'feminine' else '_male.pkl'
  styles = [s + append for s in dropdown]

  if image is None:
    img, seed = gen.generate_seed()
    images = gen.generate_art_styles(seed, styles)
    yield images

  else:

    p = projector.Projector(gen.G_ffhq)
    cropped_img = face_detector.extract_faces(image)[0]
    latent = p.run_projection(cropped_img, iterations)
    s = []
    for seed in latent:
      s = seed.cpu().numpy()
      img = gen.generate_latent(s, gen.G_ffhq)
      yield img

    styles = gen.generate_art_styles_w(s, styles)
    yield styles

    
    

demo = gr.Interface(
    fn=gen_image,
    inputs=[
        gr.Image(type="pil"),
        gr.Radio(["feminine", "masculine"], value="feminine"),
        gr.Slider(1, 1000, label='amount of iterations', value=500),
        gr.Dropdown(
            ['digital_art', 'fire_emblem', 'naruto', 'one_piece', 'painting', 'ghibli'], 
            label=['Choose your art styles'], 
            value=['digital_art', 'fire_emblem', 'naruto', 'one_piece', 'painting', 'ghibli'], multiselect=True)
    ],
    outputs=[
        "image"
    ]
)

demo.queue(concurrency_count=5, max_size=20)

demo.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://2b1d6c47-96f4-4e8f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2b1d6c47-96f4-4e8f.gradio.live
